<a href="https://colab.research.google.com/github/google-research/skai/blob/skai-colab-0000005/src/colab/Load_SKAI_Colab_MaxarImagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load SKAI Colab Maxar Imagery
This notebook aims to facilitate the loading, visuqlization, sorting, selection, filtering and downloading of the satelite imagery open sourced by [Maxar's Open Data Program](https://www.maxar.com/open-data), open data for select sudden onset major crisis events.

We leverage on the open source python library [Leafmap](https://leafmap.org/notebooks/67_maxar_open_data/) that offers functions to retrieve and dowload pre- and post-disaster imarey collection provided by Maxar.

**Before running this Colab notebook, we recommend to initialize your kernel using [Initialize SKAI Colab Kernel Notebook](https://github.com/google-research/skai/blob/main/src/colab/Initialize_SKAI_Colab_Kernel.ipynb).**


To use:
1. Open this notebook in Colab.
2. Connect to the custom kernel you want to initialize.
3. Change settings as appropriate.
4. Run all code cells.

In [44]:
#@title Settings - Run this cell first

!pip install -U --q leafmap geopandas cogeo-mosaic
!pip install --q ipywidgets

# To enable `ipywidgets`
!jupyter nbextension enable --py widgetsnbextension

# To load libraries
import pandas as pd
import datetime
import numpy as np
import os
from datetime import timezone

import leafmap
import leafmap.foliumap as leafmap_fol
import geopandas as gpd
from shapely.ops import unary_union

import ipywidgets as widgets
from IPython.display import display

# To define function for widgets
global_collections = None
global_date = None
global_aoi = None
global_m = None
allfootprint = None
pre_allfootprint = None
post_allfootprint = None
gdf = None

collection_list = leafmap.maxar_collections()

collection_picker = widgets.Dropdown(options=collection_list,
                                     value=collection_list[0])
out_colpick = widgets.Output()
disaster_date = widgets.DatePicker(description='Select disaster date',
                                   style={'description_width': 'initial'})
out_datepick = widgets.Output()
text_aoi = widgets.Text(description="Enter path to geojson aoi file (colab path or gsutil URI) ",
                        style={'description_width': 'initial'},
                        layout = widgets.Layout(width='800px'))
out_txtaoi = widgets.Output()
btn_aoi = widgets.Button(description='Upload AOI')
out_btnaoi = widgets.Output()


global_collections = leafmap.maxar_child_collections(collection_picker.value)
gdf=gpd.read_file(leafmap.maxar_collection_url(collection_picker.value,dtype='geojson'))
with out_colpick:
  display(f"The number of collections: {len(set(global_collections))} - First date: {min(gdf.datetime)}, Last date: {max(gdf.datetime)}")
gdf['geometry_maxar']=gdf.geometry
global_m = leafmap_fol.Map()
if global_date is not None:
  gdf['disaster_phase']=['pre_disaster' if x <global_date else 'post_disaster' for x in gdf.datetime]
  post_allfootprint=gpd.GeoDataFrame({'id':[1],'geometry':[gdf.loc[gdf.disaster_phase=='post_disaster'].unary_union]})
  pre_allfootprint=gpd.GeoDataFrame({'id':[1],'geometry':[gdf.loc[gdf.disaster_phase=='pre_disaster'].unary_union]})
  global_m.add_gdf(pre_allfootprint,layer_name="All Imagery Pre-disaster Footprint", style = {'color': 'green','weight': 2,'fillOpacity': 0.1})
  global_m.add_gdf(post_allfootprint,layer_name="All Imagery Post-disaster Footprint", style = {'color': 'red','weight': 2,'fillOpacity': 0.1})
else :
  allfootprint=gpd.GeoDataFrame({'id':[1],'geometry':[gdf.unary_union]})
  global_m.add_gdf(allfootprint,layer_name="All Imagery Disaster Footprint", style = {'color': 'black','weight': 2,'fillOpacity': 0.1})

with out_btnaoi:
  display(global_m)


def select_dropdown(change):
  global global_collections
  global gdf
  global global_date
  out_colpick.clear_output()
  global_collections = leafmap.maxar_child_collections(change.new)
  gdf=gpd.read_file(leafmap.maxar_collection_url(collection_picker.value,dtype='geojson'))
  with out_colpick:
    display(f"The number of collections: {len(set(global_collections))} - First date: {min(gdf.datetime)}, Last date: {max(gdf.datetime)}")

  out_datepick.clear_output()
  global_date = None
  global allfootprint
  global pre_allfootprint
  global post_allfootprint
  global global_m
  out_btnaoi.clear_output()
  gdf['geometry_maxar']=gdf.geometry
  global_m = leafmap_fol.Map()

  if global_date is not None:
    gdf['disaster_phase']=['pre_disaster' if x <global_date else 'post_disaster' for x in gdf.datetime]
    post_allfootprint=gpd.GeoDataFrame({'id':[1],'geometry':[gdf.loc[gdf.disaster_phase=='post_disaster'].unary_union]})
    pre_allfootprint=gpd.GeoDataFrame({'id':[1],'geometry':[gdf.loc[gdf.disaster_phase=='pre_disaster'].unary_union]})
    global_m.add_gdf(pre_allfootprint,layer_name="All Imagery Pre-disaster Footprint", style = {'color': 'green','weight': 2,'fillOpacity': 0.1})
    global_m.add_gdf(post_allfootprint,layer_name="All Imagery Post-disaster Footprint", style = {'color': 'red','weight': 2,'fillOpacity': 0.1})
  else :
    allfootprint=gpd.GeoDataFrame({'id':[1],'geometry':[gdf.unary_union]})
    global_m.add_gdf(allfootprint,layer_name="All Imagery Disaster Footprint", style = {'color': 'black','weight': 2,'fillOpacity': 0.1})

  if (global_aoi is not None) & (global_aoi!=''):
    if global_aoi[:5]=='gs://':
      os.system(f"""gsutil cp {global_aoi} /content/footprint.geojson""")
      gdf_aoi=gpd.read_file('/content/footprint.geojson')
    else:
      gdf_aoi=gpd.read_file(global_aoi)
    global_m.add_gdf(gdf_aoi, layer_name="Area Of Interest", style = {'color': 'orange','weight': 2,'fillOpacity': 0.05},)

  with out_btnaoi:
    display(global_m)


def select_date(change):
  global global_date
  out_datepick.clear_output()
  if change.new is None:
    global_date=None
  else:
    global_date=pd.to_datetime(change.new, errors = 'coerce').replace(tzinfo=timezone.utc)

  global gdf
  global allfootprint
  global pre_allfootprint
  global post_allfootprint
  global global_m
  out_btnaoi.clear_output()
  gdf=gpd.read_file(leafmap.maxar_collection_url(collection_picker.value,dtype='geojson'))
  gdf['geometry_maxar']=gdf.geometry
  global_m = leafmap_fol.Map()

  if global_date is not None:
    gdf['disaster_phase']=['pre_disaster' if x <global_date else 'post_disaster' for x in gdf.datetime]
    post_allfootprint=gpd.GeoDataFrame({'id':[1],'geometry':[gdf.loc[gdf.disaster_phase=='post_disaster'].unary_union]})
    pre_allfootprint=gpd.GeoDataFrame({'id':[1],'geometry':[gdf.loc[gdf.disaster_phase=='pre_disaster'].unary_union]})
    global_m.add_gdf(pre_allfootprint,layer_name="All Imagery Pre-disaster Footprint", style = {'color': 'green','weight': 2,'fillOpacity': 0.1})
    global_m.add_gdf(post_allfootprint,layer_name="All Imagery Post-disaster Footprint", style = {'color': 'red','weight': 2,'fillOpacity': 0.1})
  else :
    allfootprint=gpd.GeoDataFrame({'id':[1],'geometry':[gdf.unary_union]})
    global_m.add_gdf(allfootprint,layer_name="All Imagery Disaster Footprint", style = {'color': 'black','weight': 2,'fillOpacity': 0.1})

  if (global_aoi is not None) & (global_aoi!=''):
    if global_aoi[:5]=='gs://':
      os.system(f"""gsutil cp {global_aoi} /content/footprint.geojson""")
      gdf_aoi=gpd.read_file('/content/footprint.geojson')
    else:
      gdf_aoi=gpd.read_file(global_aoi)
    global_m.add_gdf(gdf_aoi, layer_name="Area Of Interest", style = {'color': 'orange','weight': 2,'fillOpacity': 0.05},)

  with out_btnaoi:
    display(global_m)

def select_aoi(change):
  global global_aoi
  out_txtaoi.clear_output()
  global_aoi=change.new

def button_download(change):
  global gdf
  global allfootprint
  global pre_allfootprint
  global post_allfootprint
  global global_m
  out_btnaoi.clear_output()
  gdf=gpd.read_file(leafmap.maxar_collection_url(collection_picker.value,dtype='geojson'))
  gdf['geometry_maxar']=gdf.geometry
  global_m = leafmap_fol.Map()

  if global_date is not None:
    gdf['disaster_phase']=['pre_disaster' if x <global_date else 'post_disaster' for x in gdf.datetime]
    post_allfootprint=gpd.GeoDataFrame({'id':[1],'geometry':[gdf.loc[gdf.disaster_phase=='post_disaster'].unary_union]})
    pre_allfootprint=gpd.GeoDataFrame({'id':[1],'geometry':[gdf.loc[gdf.disaster_phase=='pre_disaster'].unary_union]})
    global_m.add_gdf(pre_allfootprint,layer_name="All Imagery Pre-disaster Footprint", style = {'color': 'green','weight': 2,'fillOpacity': 0.1})
    global_m.add_gdf(post_allfootprint,layer_name="All Imagery Post-disaster Footprint", style = {'color': 'red','weight': 2,'fillOpacity': 0.1})
  else :
    allfootprint=gpd.GeoDataFrame({'id':[1],'geometry':[gdf.unary_union]})
    global_m.add_gdf(allfootprint,layer_name="All Imagery Disaster Footprint", style = {'color': 'black','weight': 2,'fillOpacity': 0.1})

  if (global_aoi is not None) & (global_aoi!=''):
    if global_aoi[:5]=='gs://':
      os.system(f"""gsutil cp {global_aoi} /content/footprint.geojson""")
      gdf_aoi=gpd.read_file('/content/footprint.geojson')
    else:
      gdf_aoi=gpd.read_file(global_aoi)
    global_m.add_gdf(gdf_aoi, layer_name="Area Of Interest", style = {'color': 'orange','weight': 2,'fillOpacity': 0.05},)

  with out_btnaoi:
    display(global_m)

collection_picker.observe(select_dropdown, names="value")
disaster_date.observe(select_date, names="value")
text_aoi.observe(select_aoi, names="value")
btn_aoi.on_click(button_download)

# To define function for selection
def function_logit(x):
  return 2*(1/(1+np.exp(-0.1*x)))-1

def function_score_final(x):
  x['score_tile'] = x['overlay_pre_post']*((100 - x['tile:clouds_percent_pre'])/100)*(1-function_logit((global_date-x['datetime_pre']).dt.days/365.25))
  x['score_col'] = x['overlay_prec_post']*((100 - x['tile:clouds_percent_prec'])/100)*(1-function_logit((global_date-x['datetime_pre']).dt.days/365.25))
  return x

def recursive_function(group):
  result = []
  intermediate = []
  area =[]
  idx=0
  for index, row in group.iterrows():
    if idx==0:
      idx+=1
      list_= row['catalog_id_pre']
      result.append(row['geometry_maxar_post'].difference(row['geometry_prec_post']))
      intermediate.append(1)
      area.append(result[-1].area/row['geometry_maxar_post'].area)
    else:
      if row['catalog_id_pre'] == list_ :
        result.append(result[-1])
        intermediate.append(intermediate[-1])
        area.append(area[-1])
      else:
        list_=[row['catalog_id_pre'],row['quadkey_pre']]
        if area[-1]<=1-row['area_to_cover']:
          result.append(result[-1])
          intermediate.append(0)
          area.append(area[-1])
        else:
          diff = result[-1].difference(row['geometry_prec_post'])
          if diff.is_empty:
            result.append(result[-1])
            intermediate.append(0)
            area.append(area[-1])
          else:
            result.append(diff)
            intermediate.append(1)
            area.append(result[-1].area/row['geometry_maxar_post'].area)
  return [group.catalog_id_pre,group.quadkey_pre,result, intermediate, area]

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [47]:
#@title Disaster Information
display(collection_picker)
display(out_colpick)
display(disaster_date)
display(text_aoi,btn_aoi)
display(out_btnaoi)

Dropdown(options=('BayofBengal-Cyclone-Mocha-May-23', 'Emilia-Romagna-Italy-flooding-may23', 'Gambia-flooding-…

Output()

DatePicker(value=None, description='Select disaster date', style=DescriptionStyle(description_width='initial')…

Text(value='', description='Enter path to geojson aoi file (colab path or gsutil URI) ', layout=Layout(width='…

Button(description='Upload AOI', style=ButtonStyle())

Output()

In [48]:
# @title Automatic selection and visualisation of the intersected pre- and post-disaster footprint

# We first join all the pre and post imagery that intersect in the aoi
if not (global_aoi=='' or global_aoi is None) :
  gdf_aoi=gpd.read_file(global_aoi)
  intersecting_aoi_all = gdf.sjoin(gdf_aoi, how='inner')#Find the polygons that intersect.
else:
  intersecting_aoi_all = gdf
intersection_final=intersecting_aoi_all.loc[(intersecting_aoi_all.disaster_phase=='pre_disaster')].sjoin(intersecting_aoi_all.loc[(intersecting_aoi_all.disaster_phase=='post_disaster')], how='inner',lsuffix='pre',rsuffix='post')

# We then compute measures and geometries related to area intersected and cloud coverage for each pre-disaster tile and collection
intersect=intersection_final['geometry_maxar_post'].intersection(intersection_final['geometry_maxar_pre']).area
intersection_final['overlay_pre_post']=intersect/intersection_final['geometry_maxar_post'].area #percentage of pre-disaster tile overlap over post-disaster tile

intersection_final['geometry_prec_post']=intersection_final['geometry_maxar_post'].intersection(intersection_final.groupby(['catalog_id_pre','catalog_id_post','quadkey_post'])['geometry_maxar_pre'].transform(lambda x : unary_union(x)))
intersect=intersection_final['geometry_maxar_post'].intersection(intersection_final['geometry_prec_post']).area
intersection_final['overlay_prec_post']=intersect/intersection_final['geometry_maxar_post'].area
intersection_final['max_overlay_prec_post']=intersection_final.groupby(['catalog_id_post','quadkey_post'])['overlay_prec_post'].transform('max')

intersection_final['tile:clouds_percent_prec']=intersection_final['tile:clouds_percent_pre'].values*intersection_final['overlay_pre_post'].values/intersection_final['overlay_prec_post'].values
intersection_final['tile:clouds_percent_prec']=np.minimum(100,intersection_final.groupby(['catalog_id_pre','catalog_id_post','quadkey_post'])['tile:clouds_percent_prec'].transform('sum'))

# We compute the selection score, based on coverage with intersected post-disaster tile, cloud coverage and time difference, for each pre-disaster tile and collection
intersection_final=function_score_final(intersection_final)

# We select the tiles and collection based on the maximum coverage and best score
test=intersection_final
test=pd.merge(test,test.groupby(['catalog_id_post','quadkey_post'])['geometry_prec_post','geometry_maxar_post'].apply(lambda x : unary_union(x['geometry_prec_post']).area/x['geometry_maxar_post'].iloc[0].area).reset_index(name='area_to_cover'),on=['catalog_id_post','quadkey_post'])
test.sort_values(by=['catalog_id_post','quadkey_post','score_col','catalog_id_pre'], ascending=[True,True,False,True], inplace=True)
results = test.groupby(['catalog_id_post','quadkey_post']).apply(recursive_function)
test_=pd.DataFrame(results.reset_index()[0].to_list(), index= results.index, columns=['catalog_id_pre','quadkey_pre','poly_diff','select_bool','area_perc_diff']).explode(['catalog_id_pre','quadkey_pre','poly_diff','select_bool','area_perc_diff']).reset_index()
test_=test_.loc[test_.select_bool==1]
test=pd.merge(test,test_[['catalog_id_post','quadkey_post','catalog_id_pre','quadkey_pre']],on=['catalog_id_post','quadkey_post','catalog_id_pre','quadkey_pre'], how='inner')

gdf_final=test
gdf_final['date_pre']=pd.to_datetime(gdf_final['datetime_pre'])
gdf_final['date_pre']=gdf_final['date_pre'].dt.strftime('%Y%m%d')
gdf_final['date_post']=pd.to_datetime(gdf_final['datetime_post'])
gdf_final['date_post']=gdf_final['date_post'].dt.strftime('%Y%m%d')

# Analyse of number of tiles filter out
intital_pre=intersecting_aoi_all.loc[intersecting_aoi_all.disaster_phase=='pre_disaster',['catalog_id','quadkey','geometry']].drop_duplicates(keep='first')
print(f"Total of {len(intital_pre)} pre-disaster imagery tiles initially intersecting AOI")
intital_post=intersecting_aoi_all.loc[intersecting_aoi_all.disaster_phase=='post_disaster',['catalog_id','quadkey','geometry']].drop_duplicates(keep='first')
print(f"Total of {len(intital_post)} post-disaster imagery tiles initially intersecting AOI")
final_pre=gdf_final[['catalog_id_pre','quadkey_pre','geometry_maxar_pre','date_pre','visual_pre']].drop_duplicates(keep='first').set_geometry("geometry_maxar_pre")
print(f"Total of {len(final_pre)} pre-disaster imagery tiles finaly selected intersecting AOI and post-disaster imagery tiles")
final_post=gdf_final[['catalog_id_post','quadkey_post','geometry_maxar_post','date_post','visual_post']].drop_duplicates(keep='first').set_geometry("geometry_maxar_post")
print(f"Total of {len(final_post)} post-disaster imagery tiles finaly selected intersecting AOI and pre-disaster imagery tiles")

post_footprint=gpd.GeoDataFrame({'id':[1],'geometry':[final_post.unary_union]})
pre_footprint=gpd.GeoDataFrame({'id':[1],'geometry':[final_pre.unary_union]})

if not (global_aoi=='' or global_aoi is None) :
  intersection_pre_post_aoi=gpd.GeoDataFrame(post_footprint.intersection(pre_footprint).intersection(gdf_aoi)).set_geometry(0)
else:
  intersection_pre_post_aoi=gpd.GeoDataFrame(post_footprint.intersection(pre_footprint)).set_geometry(0)

# @title Visualize selected tiles footprints
m = leafmap_fol.Map()
if not (global_aoi=='' or global_aoi is None):
  m.add_gdf(gdf_aoi, layer_name="Area Of Interest", style = {'color': 'orange','weight': 2,"fill": False },)
m.add_gdf(pre_footprint,layer_name="AOI Imagery Pre-disaster", style = {'color': 'green','weight': 2,"fill": False})
m.add_gdf(post_footprint,layer_name="AOI Imagery Post-disaster", style = {'color': 'red','weight': 2,"fill": False})
m.add_gdf(intersection_pre_post_aoi, layer_name="AOI Imagery Overlap", style = {'color': 'blue','weight': 2,"fill": False})

for id in pd.unique(final_pre.catalog_id_pre):
  style = {
  'fillColor': 'green',     # Fill color of polygons
  'color': 'green',       # Border color of polygons
  'weight': 0.1,            # Border thickness
  'fillOpacity': 0.1      # Opacity of the fill color
  }
  m.add_gdf(final_pre.loc[final_pre.catalog_id_pre==id], layer_name=f"{id}_pre",style=style,info_mode='on_click')

for id in pd.unique(final_post.catalog_id_post):
  style = {
  'fillColor': 'red',     # Fill color of polygons
  'color': 'red',       # Border color of polygons
  'weight': 0.1,            # Border thickness
  'fillOpacity': 0.1,      # Opacity of the fill color
  }
  m.add_gdf(final_post.loc[final_post.catalog_id_post==id], layer_name=f"{id}_post",style=style,info_mode='on_click')

m

Total of 8239 pre-disaster imagery tiles initially intersecting AOI
Total of 485 post-disaster imagery tiles initially intersecting AOI
Total of 1229 pre-disaster imagery tiles finaly selected intersecting AOI and post-disaster imagery tiles
Total of 454 post-disaster imagery tiles finaly selected intersecting AOI and pre-disaster imagery tiles


In [49]:
# @title Visualize selected pre- and post-disaster tile images

post_list=pd.unique(final_post[['date_post', 'catalog_id_post']].agg('_'.join, axis=1))
post_picker = widgets.Dropdown(description='Select post-disaster imagery',style={'description_width': 'initial'},
                               layout = widgets.Layout(width='400px'),
                               options=post_list,
                               value=post_list[0])

pre_list=pd.unique(gdf_final.loc[gdf_final.catalog_id_post==post_list[0].split('_')[1],['date_pre','catalog_id_pre']].agg('_'.join, axis=1))
pre_picker = widgets.Dropdown(description='Select pre-disaster imagery',style={'description_width': 'initial'},
                              layout = widgets.Layout(width='400px'),
                              options=pre_list,
                              value=pre_list[0])

post_print=gpd.GeoDataFrame({'id':[1],'geometry':[final_post.loc[final_post.catalog_id_post==post_list[0].split('_')[1]].unary_union]})
pre_print=gpd.GeoDataFrame({'id':[1],'geometry':[final_pre.loc[final_pre.catalog_id_pre==pre_list[0].split('_')[1]].unary_union]})
if not (global_aoi=='' or global_aoi is None) :
  intersect_pre_post_aoi=gpd.GeoDataFrame(post_print.intersection(pre_print).intersection(gdf_aoi)).set_geometry(0)
else:
  intersect_pre_post_aoi=gpd.GeoDataFrame(post_print.intersection(pre_print)).set_geometry(0)

glob_m = None
glob_m = leafmap.Map()

out_post = widgets.Output()
out_pre = widgets.Output()
out_vis = widgets.Output()

def select_dropdown_post(change):
  global glob_m
  out_vis.clear_output()

  new_post=change.new
  pre_list=pd.unique(gdf_final.loc[gdf_final.catalog_id_post==new_post.split('_')[1],['date_pre','catalog_id_pre']].agg('_'.join, axis=1))
  pre_picker.options=pre_list
  pre_picker.value=pre_list[0]

  pre_value=pre_picker.value
  post_value=post_picker.value

  post_print=gpd.GeoDataFrame({'id':[1],'geometry':[final_post.loc[final_post.catalog_id_post==post_value.split('_')[1]].unary_union]})
  pre_print=gpd.GeoDataFrame({'id':[1],'geometry':[final_pre.loc[final_pre.catalog_id_pre==pre_value.split('_')[1]].unary_union]})
  if not (global_aoi=='' or global_aoi is None) :
    intersect_pre_post_aoi=gpd.GeoDataFrame(post_print.intersection(pre_print).intersection(gdf_aoi)).set_geometry(0)
  else:
    intersect_pre_post_aoi=gpd.GeoDataFrame(post_print.intersection(pre_print)).set_geometry(0)
  pre_stac = leafmap.maxar_tile_url(collection_picker.value, pre_value.split('_')[1], dtype='json')
  post_stac = leafmap.maxar_tile_url(collection_picker.value, post_value.split('_')[1], dtype='json')

  glob_m = leafmap.Map()
  if not (global_aoi=='' or global_aoi is None) :
    glob_m.add_gdf(gdf_aoi, layer_name="Area Of Interest", style = {'color': 'orange','weight': 2,"fill": False },)
  glob_m.add_gdf(intersection_pre_post_aoi, layer_name="AOI Imagery Overlap", style = {'color': 'blue','weight': 2,"fill": False})
  glob_m.add_gdf(intersect_pre_post_aoi, layer_name="AOI Imagery Overlap Collection", style = {'color': 'cyan','weight': 2,"fill": False})
  glob_m.split_map(
    left_layer=pre_stac,
    right_layer=post_stac,
    left_label=f'Pre-event_{pre_list[0]}',
    right_label=f'Post-event_{post_picker.value}',)

def select_dropdown_pre(change):
  global glob_m
  out_vis.clear_output()

  new_pre=change.new
  post_value=post_picker.value

  post_print=gpd.GeoDataFrame({'id':[1],'geometry':[final_post.loc[final_post.catalog_id_post==post_value.split('_')[1]].unary_union]})
  pre_print=gpd.GeoDataFrame({'id':[1],'geometry':[final_pre.loc[final_pre.catalog_id_pre==new_pre.split('_')[1]].unary_union]})
  if not (global_aoi=='' or global_aoi is None) :
    intersect_pre_post_aoi=gpd.GeoDataFrame(post_print.intersection(pre_print).intersection(gdf_aoi)).set_geometry(0)
  else:
    intersect_pre_post_aoi=gpd.GeoDataFrame(post_print.intersection(pre_print)).set_geometry(0)
  pre_stac = leafmap.maxar_tile_url(collection_picker.value, new_pre.split('_')[1], dtype='json')
  post_stac = leafmap.maxar_tile_url(collection_picker.value, post_value.split('_')[1], dtype='json')

  glob_m = leafmap.Map()
  if not (global_aoi=='' or global_aoi is None) :
    glob_m.add_gdf(gdf_aoi, layer_name="Area Of Interest", style = {'color': 'orange','weight': 2,"fill": False },)
  glob_m.add_gdf(intersection_pre_post_aoi, layer_name="AOI Imagery Overlap", style = {'color': 'blue','weight': 2,"fill": False})
  glob_m.add_gdf(intersect_pre_post_aoi, layer_name="AOI Imagery Overlap Collection", style = {'color': 'cyan','weight': 2,"fill": False})
  glob_m.split_map(
    left_layer=pre_stac,
    right_layer=post_stac,
    left_label=f'Pre-event_{new_pre}',
    right_label=f'Post-event_{post_picker.value}',)

  with out_vis:
    display(glob_m)


post_picker.observe(select_dropdown_post, names="value")
pre_picker.observe(select_dropdown_pre, names="value")

pre_value=pre_picker.value
post_value=post_picker.value
pre_stac = leafmap.maxar_tile_url(collection_picker.value, pre_value.split('_')[1], dtype='json')
post_stac = leafmap.maxar_tile_url(collection_picker.value, post_value.split('_')[1], dtype='json')

glob_m = leafmap.Map()

if not (global_aoi=='' or global_aoi is None) :
  glob_m.add_gdf(gdf_aoi, layer_name="Area Of Interest", style = {'color': 'orange','weight': 2,"fill": False },)
glob_m.add_gdf(intersection_pre_post_aoi, layer_name="AOI Imagery Overlap", style = {'color': 'blue','weight': 2,"fill": False})
glob_m.add_gdf(intersect_pre_post_aoi, layer_name="AOI Imagery Overlap Collection", style = {'color': 'cyan','weight': 2,"fill": False})

glob_m.split_map(
  left_layer=pre_stac,
  right_layer=post_stac,
  left_label=f'Pre-event_{pre_picker.value}',
  right_label=f'Post-event_{post_picker.value}',)

with out_post:
  display(post_picker)
with out_pre:
  display(pre_picker)
with out_vis:
  display(glob_m)

display(out_post)
display(out_pre)
display(out_vis)

Output()

Output()

Output()

In [ ]:
# @title Launch the download of the selected imagery from Maxar bucket

import json
import shutil
from typing import Sequence
import urllib.parse

from absl import app
from absl import flags
from bs4 import BeautifulSoup
import requests
import tensorflow as tf
from tqdm import tqdm

from google.cloud import storage

pathsys_credentials = '/root/service-account-private-key.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = pathsys_credentials

##@markdown ---
#@markdown Please enter the path of destination folder to store the imagery (colab path or gsutil URI):
PATH_FILE_LIST = "" #@param {type:"string"}
PATH_FILE_AOI = "" #@param {type:"string"}
PATH_FOLDER_PRE = "" #@param {type:"string"}
PATH_FOLDER_POST = "" #@param {type:"string"}

def check_file_exists(bucket_name, file_name):
    storage_client = storage.Client.from_service_account_json(pathsys_credentials)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name.strip("/"))
    return blob.exists()

def download_image(url: str, output_path: str) -> None:
  with requests.get(url, stream=True) as r:
    with tf.io.gfile.GFile(output_path, 'wb') as f:
      shutil.copyfileobj(r.raw, f)

def make_download_path(url: str, output_dir: str) -> str:
  parsed_url = urllib.parse.urlparse(url)
  components = parsed_url.path.split('/')
  image_name = components[-1].split('.')[0]
  return f'{output_dir}/{image_name}-{components[-3]}.tif'

def download_images(urls: Sequence[str], output_dir: str) -> None:
  if output_dir[:5]!='gs://':
    for url in tqdm(urls, total=len(urls)):
      print(make_download_path(url, output_dir))
      download_image(url, make_download_path(url, output_dir))
  else:
    bucket=output_dir.split('/')[2]
    output_dir='/'+'/'.join(output_dir.split('/')[3:])
    counter_k=0
    for url in tqdm(urls, total=len(urls)):
      if check_file_exists(bucket, make_download_path(url, output_dir)):
        print(make_download_path(url, f'gs://{bucket}{output_dir}'),f'file already existing')
      else:
        counter_k+=1
        download_image(url, make_download_path(url, '/content/sample_data'))
        os.system(f"""gsutil cp {make_download_path(url, '/content/sample_data')} {make_download_path(url, f'gs://{bucket}{output_dir}')}""")
        print(make_download_path(url, f'gs://{bucket}{output_dir}'))
      if counter_k>20:
        counter_k=0
        os.system(f"""rm /content/sample_data/*""")
    os.system(f"""rm /content/sample_data/*""")

if PATH_FILE_LIST!="":
  if PATH_FILE_LIST[:5]!='gs://':
    with open(PATH_FILE_LIST, 'w') as file:
        gdf_final.to_csv(file)
  else:
    with open('/content/sample_data/imagery_list_maxar.csv', 'w') as file:
        gdf_final.to_csv(file)
    os.system(f"""gsutil cp /content/sample_data/imagery_list_maxar.csv {PATH_FILE_LIST}""")

if PATH_FILE_AOI!="":
  if PATH_FILE_AOI[:5]!='gs://':
    with open(PATH_FILE_AOI, 'w') as file:
        file.write(intersection_pre_post_aoi.to_json())
  else:
    with open('/content/sample_data/aoi_intersect_maxar.geojson', 'w') as file:
        file.write(intersection_pre_post_aoi.to_json())
    os.system(f"""gsutil cp /content/sample_data/aoi_intersect_maxar.geojson {PATH_FILE_AOI}""")

print('Dowload of the pre-disaster imagery')
download_images(final_pre.visual_pre, PATH_FOLDER_PRE)

print('Dowload of the post-disaster imagery')
download_images(final_post.visual_post, PATH_FOLDER_POST)